In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from itertools import chain

In [2]:
def scrape_website(number):
    url = 'https://ofdsearch.doc.nv.gov/'  # Replace with the actual URL

    # Send the number as form-data
    payload = {'onumber': number}
    response = requests.post(url, data=payload, verify=False)

    # Parse the HTML response
    soup = BeautifulSoup(response.text, 'html5lib')
    
    soup.prettify()

    # Find the caption with the desired text
    caption = soup.find('caption', text='Identification and Demographics')
    
    data_row = caption.find_next('tbody').find_all('tr')[1]  # Assuming the data row is at index 1

    # Extract the required information from the row
    details = {
        'Name': data_row.find_all('td')[0].text,
        'Offender ID': data_row.find_all('td')[1].text,
        'Gender': data_row.find_all('td')[2].text,
        'Ethnic': data_row.find_all('td')[3].text,
        'Age': data_row.find_all('td')[4].text,
        'Height': data_row.find_all('td')[5].text,
        'Weight': data_row.find_all('td')[6].text,
        'Build': data_row.find_all('td')[7].text,
        'Complexion': data_row.find_all('td')[8].text,
        'Hair': data_row.find_all('td')[9].text,
        'Eyes': data_row.find_all('td')[10].text,
        'Institution': data_row.find_all('td')[11].text,
        'Custody Level': data_row.find_all('td')[12].text,
        'Aliases': data_row.find_all('td')[13].text,
        'Prior Felonies': data_row.find_all('td')[14].text
    }
    
    # Find the <caption> element containing "Booking Information"
    caption = soup.find('caption', text='Booking Information')

    # Find the parent <tr> element of the <caption>
    tr_caption = caption.find_next('tbody').find_all('tr')
    
    headers = []
    for th in tr_caption[0].find_all('th'):
        headers.append(th.text)

    # Find all <tr> elements after the caption until the end of the <table>
    rows = tr_caption[1:]

    # Extract the information from each <tr> element
    details_booking = []
    details_booking.append(headers)
    for row in rows:
        detail = row.find_all('td')
        database = []
        for data in detail:
            database.append(data.text)
        details_booking.append(database)
        
    details['Booking Information'] = details_booking

    return details


In [3]:
def scrape_make_df(onnumber):
    number_to_send = onnumber
    details = scrape_website(number_to_send)
    df1 = pd.DataFrame(
        {
            'Name': [details['Name']],
            'Offender ID': [details['Offender ID']],
            'Gender': [details['Gender']],
            'Ethnic': [details['Ethnic']],
            'Age': [details['Age']],
            'Height': [details['Height']],
            'Weight': [details['Weight']],
            'Build': [details['Build']],
            'Complexion': [details['Complexion']],
            'Hair': [details['Hair']],
            'Eyes': [details['Eyes']],
            'Institution': [details['Institution']],
            'Custody Level': [details['Custody Level']],
            'Aliases': [details['Aliases']],
            'Prior Felonies': [details['Prior Felonies']],
        },
        index=[0]  # Provide a list of indices with length equal to the data length
    )

    # Create a DataFrame from the arrays in the "Booking Information" values
    booking_data = details['Booking Information']
    headers = booking_data[0]
    booking_data = booking_data[1:]
    num_rows = len(booking_data)  # Number of rows in booking data
    offender_id_column = [details['Offender ID']] * num_rows  # Repeat the offender ID for each row
    df2 = pd.DataFrame(booking_data, columns=headers)
    df2['Offender ID'] = offender_id_column  # Add the 'Offender ID' column
    
    return df1, df2

In [4]:
main_bio = pd.DataFrame()
main_act = pd.DataFrame()

In [6]:
for onnumber in range(1198590,1198690):
    try:
        df1, df2 = scrape_make_df(onnumber)
        main_bio = main_bio.append(df1)
        main_act = main_act.append(df2)
    except:
        continue

/Users/Abhishek/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ofdsearch.doc.nv.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/var/folders/pd/rs70nkb90fl902tbtrpys37w0000gn/T/ipykernel_92413/3248739181.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_bio = main_bio.append(df1)
/var/folders/pd/rs70nkb90fl902tbtrpys37w0000gn/T/ipykernel_92413/3248739181.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_act = main_act.append(df2)
/Users/Abhishek/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ofdsea

In [7]:
main_bio.head()

,Name,Offender ID,Gender,Ethnic,Age,Height,Weight,Build,Complexion,Hair,Eyes,Institution,Custody Level,Aliases,Prior Felonies
0,ROMAN A CALDERON,1198590,Male,HISPANIC OR LATINO,29,"5'9""",145lb,,MEDIUM,BROWN,BROWN,INACTIVE-DSCHREL,,"CALDERON ERIC JIMENEZ, JOKER, CONEJO, ROMA...",NO
0,ALEJANDRO SORIA-BAUTISTA,1198591,Male,HISPANIC OR LATINO,46,"5'6""",140lb,,MEDIUM,BROWN,BROWN,INACTIVE-DSCHREL,,,NO
0,RYAN JAMES STONE,1198592,Male,CAUCASIAN,43,"5'9""",190lb,,FAIR,BROWN,GREEN,INACTIVE-DSCHREL,,RYAN STONE,YES
0,DOMINEAQ WILSON,1198593,Male,NOT HISPANIC,24,"6'3""",175lb,,DARK,BLACK,BROWN,PAROLE,UNASSIGNED,"DOMINEAQ M WILSON, DOMINEAQ MARCHELL-THOMAS WI...",YES
0,JABAR ALLEN,1198594,Male,NOT HISPANIC,24,"5'10""",180lb,,DARK,BLACK,BROWN,INACTIVE-DSCHREL,,"JABAR KENTAE ALLEN, JABAR J ALLEN, JB, JABAR...",NO


In [8]:
main_act.head()

,Offense Code,Offense Description,Sent. Status,Sent. Min,Sent. Max,Sent. PED,Sent. MPR,Sent. County,Sent. PEXD,Sent. Type,Sent. RRD,Sent. Start Date,Offender ID
0,2144,ABUSE/NEGLECT/ENDANGER CHILD,Aggregated,0 yr. 18 mo. 0 days,0 yr. 48 mo. 0 days,,,CLARK COUNTY COURTHOUSE,,DETERMINATE,,2017-12-18,1198590
1,A008,Aggregate,Inactive,3 yr. 0 mo. 0 days,8 yr. 0 mo. 0 days,2020-12-17,,AGGREGATE SENTENCING,2022-04-12,DETERMINATE,,2017-12-18,1198590
2,2144,ABUSE/NEGLECT/ENDANGER CHILD,Aggregated,0 yr. 18 mo. 0 days,0 yr. 48 mo. 0 days,,,CLARK COUNTY COURTHOUSE,,DETERMINATE,,2020-02-12,1198590
0,2297,ATT LEWDNESS W/ A MINOR,Inactive,0 yr. 36 mo. 0 days,0 yr. 120 mo. 0 days,2019-05-25,,CLARK COUNTY COURTHOUSE,2022-02-26,DETERMINATE,,2016-05-26,1198591
0,299,BURGLARY,Inactive,0 yr. 16 mo. 0 days,0 yr. 72 mo. 0 days,2019-05-05,,CLARK COUNTY COURTHOUSE,2021-06-13,DETERMINATE,,2018-01-06,1198592


In [9]:
main_bio.to_excel('Nevada_Felon_Data_Bio.xlsx')
main_act.to_excel('Nevada_Felon_Data_Offence.xlsx')